<a href="https://colab.research.google.com/github/nikhilkr31/bear-classifier/blob/main/bear-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Image Search API
from duckduckgo_search import DDGS

#Provides Utils classes
from fastcore.all import *

#Downloading URLs
from fastdownload import download_url

#FastAI Vision
from fastai.vision.all import *

import os

#Environment variable

%set_env PYTORCH_ENABLE_MPS_FALLBACK 1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [ ]:
#Search for images using the Duckduckgo imagesearch API
def search_images(term, max_images=150):
    return L(DDGS().images(term, max_results=max_images)).itemgot('image')

In [ ]:
# #Search and save the urls
# urls = search_images('brown bear', max_images=150)
# print(urls[0])

# #download the images from the URLs

# dest = 'bird.jpg'
# download_url(urls[0], dest, show_progress=True)

# #Preview image while resizing
# im = Image.open(dest)
# im.to_thumb(128,128)

In [ ]:
#Search and download images into their respective folders
bear_types = ['black','brown','teddy']
path = Path('bears')

from time import sleep

for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} bear'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} bear'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} bear'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
#check for failed and unlink the path

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

59

## DataLoaders

In [ ]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders):
        self.loaders = loaders
    def __getitem__(self, i):
        return self.loaders[i]
    train, valid = add_props(lambda i,self: self[i])

To turn our downloaded data into a DataLoaders object, we need 4 things:
1. What kinds of data we are working with
2. How to get the lists of items
3. How to label these items
4. How to create the validation set

In [ ]:
#define the datablock
dls = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = parent_label,
    #Using Randomized Resize crop for Data Augmentation
    item_tfms = Resize(128),
    batch_tfms=aug_transforms(mult=2)).dataloaders(path)

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


NotImplementedError: The operator 'aten::_linalg_solve_ex.result' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
dls.show_batch(max_n=6)

In [ ]:
#Adding Test comments